# Unit 1: 4 - Defining Tools for SmolLM2

**Collaborators**:
* Roberto Rodriguez ([@Cyb3rWard0g](https://x.com/Cyb3rWard0g))

## Introduction to Tools

AI Agents extend their capabilities through **Tools**, which are predefined functions that perform specific tasks beyond the model's internal knowledge. Tools allow LLMs to:
- Fetch up-to-date information
- Perform calculations
- Interact with APIs
- Retrieve external knowledge

### Example Use Cases
| **Tool**         | **Description**                                     |
|----------------|-------------------------------------------------|
| Web Search    | Fetches real-time data from the internet       |
| Calculator    | Performs arithmetic operations                 |
| API Interface | Calls external services (GitHub, Weather API)  |
| Retrieval     | Retrieves specific data from a database       |

## Defining a Simple Tool

A Tool consists of:
- A **name**
- A **description**
- Expected **arguments** and their types
- An **output type**
- A **callable function**

Here’s a basic **calculator tool** that multiplies two numbers:

In [1]:
def calculator(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

# Example usage
print("Example Calculation:", calculator(3, 4))  # Output: 12

Example Calculation: 12


## Creating a Generic Tool Class

Instead of manually writing descriptions for each tool, we can define a **Tool class** that extracts this information automatically using Python’s introspection.

In [2]:
import inspect

class Tool:
    """
    A class representing a reusable AI Tool.
    """
    def __init__(self, name: str, description: str, func: callable):
        self.name = name
        self.description = description
        self.func = func
        self.arguments = inspect.signature(func).parameters
        self.outputs = inspect.signature(func).return_annotation
    
    def to_string(self) -> str:
        """
        Generates a structured textual representation of the tool.
        """
        args_str = ", ".join([f"{arg}: {param.annotation}" for arg, param in self.arguments.items()])
        return f"Tool Name: {self.name}, Description: {self.description}, Arguments: {args_str}, Outputs: {self.outputs}"
    
    def __call__(self, *args, **kwargs):
        """Invoke the tool."""
        return self.func(*args, **kwargs)

In [3]:
# Example usage
calculator_tool = Tool("calculator", "Multiply two integers.", calculator)
print(calculator_tool.to_string())

Tool Name: calculator, Description: Multiply two integers., Arguments: a: <class 'int'>, b: <class 'int'>, Outputs: <class 'int'>


## Using a Decorator for Simplicity

To simplify tool creation, we can use a **Python decorator**:

In [4]:
# Define a decorator to register tools

def tool(func):
    """Decorator to register a function as a tool."""
    return Tool(func.__name__, func.__doc__, func)

@tool
def calculator(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

# Example usage
print(calculator.to_string())

Tool Name: calculator, Description: Multiply two integers., Arguments: a: <class 'int'>, b: <class 'int'>, Outputs: <class 'int'>


## Integrating Tools into SmolLM2 System Messages

To make SmolLM2 aware of available tools, we can include them in the **system prompt**:

### Install Required Libraries

In [5]:
# !pip install transformers torch

### Loading SmolLM2 Efficiently

To avoid downloading the model every time (**~3.42 GB**), we first check if it exists locally before loading:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

MODEL_NAME = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
MODEL_DIR = "data/smollm2"

def load_model():
    if os.path.exists(MODEL_DIR):
        print("Loading model from local directory.")
        model = AutoModelForCausalLM.from_pretrained(MODEL_DIR)
    else:
        print("Downloading model...")
        model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
        model.save_pretrained(MODEL_DIR)
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = load_model().to(device)

Loading model from local directory.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Adding Tools to System Prompt

In [7]:
# Define a system message including tool descriptions
system_message = {
    "role": "system",
    "content": "You are an assistant with access to tools. Available tools: \n"
    + calculator.to_string()
}

In [8]:
# Example user interaction
messages = [
    system_message,
    {"role": "user", "content": "What is 5 times 6?"},
]

In [9]:
# Convert messages into model-compatible format
input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(input_text)

<|im_start|>system
You are an assistant with access to tools. Available tools: 
Tool Name: calculator, Description: Multiply two integers., Arguments: a: <class 'int'>, b: <class 'int'>, Outputs: <class 'int'><|im_end|>
<|im_start|>user
What is 5 times 6?<|im_end|>
<|im_start|>assistant



In [ ]:
# Encode with attention mask
encoded_input = tokenizer(input_text, return_tensors="pt").to(device)
input_ids = encoded_input["input_ids"]
attention_mask = encoded_input["attention_mask"]  # Explicit attention mask
count_prompt_tokens = input_ids.shape[1]

# Generate response with proper settings
outputs = model.generate(
    input_ids, 
    attention_mask=attention_mask,  # Avoids padding/EOS confusion
    max_new_tokens=50,
    eos_token_id=tokenizer.eos_token_id  # Stops at <|im_end|>
)

In [20]:
# Extract only assistant-generated tokens
generated_tokens = outputs[0, count_prompt_tokens:]

# Decode assistant response
output = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("Assistant Response:", output)

Assistant Response: <tool_call>[{"name": "calculator", "arguments": {"a": 5, "b": 6}}]</tool_call>


The assistant’s response contains the tool call wrapped in `<tool_call>` tags, following this format:

`<tool_call>[{"name": "calculator", "arguments": {"a": 5, "b": 6}}]</tool_call>`

We need to extract the JSON object:

In [21]:
start_index = output.index("{")
end_index = output.rindex("}")
chosen_tool = output[start_index : end_index + 1]

In [22]:
chosen_tool

'{"name": "calculator", "arguments": {"a": 5, "b": 6}}'

Finally, we convert the tool call string into a Python dictionary:

In [23]:
import json
parsed_tool = json.loads(chosen_tool)
parsed_tool

{'name': 'calculator', 'arguments': {'a': 5, 'b': 6}}

### Final Updated Block Code

In [25]:
# Define a system message including tool descriptions
system_message = {
    "role": "system",
    "content": "You are an assistant with access to tools. Available tools: \n"
    + calculator.to_string()
}

# Example user interaction
messages = [
    system_message,
    {"role": "user", "content": "What is 5 times 6?"},
]

# Convert messages into model-compatible format
input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print("Input Text:\n", input_text)

# Encode input with attention mask
encoded_input = tokenizer(input_text, return_tensors="pt").to(device)
input_ids = encoded_input["input_ids"]
attention_mask = encoded_input["attention_mask"]
count_prompt_tokens = input_ids.shape[1]  # Save prompt length

# Generate response
outputs = model.generate(
    input_ids, 
    attention_mask=attention_mask,
    max_new_tokens=50,
    eos_token_id=tokenizer.eos_token_id
)

# Extract only assistant-generated tokens
generated_tokens = outputs[0, count_prompt_tokens:]

# Decode assistant response
output = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("Assistant Response:\n", output)

# Extract tool invocation
start_index = output.index("{")
end_index = output.rindex("}")
chosen_tool = output[start_index : end_index + 1]

# Parse tool call into JSON
import json
parsed_tool = json.loads(chosen_tool)

print("\nParsed Tool Call:\n", parsed_tool)

Input Text:
 <|im_start|>system
You are an assistant with access to tools. Available tools: 
Tool Name: calculator, Description: Multiply two integers., Arguments: a: <class 'int'>, b: <class 'int'>, Outputs: <class 'int'><|im_end|>
<|im_start|>user
What is 5 times 6?<|im_end|>
<|im_start|>assistant

Assistant Response:
 <tool_call>[{"name": "calculator", "arguments": {"a": 5, "b": 6}}]</tool_call>

Parsed Tool Call:
 {'name': 'calculator', 'arguments': {'a': 5, 'b': 6}}


### Executing Tools
Now that we have extracted and parsed the tool invocation, the next step is to execute the tool. The assistant generates the tool call, but it is up to us to process and execute it.

We follow these steps:

1. Retrieve the tool name and arguments from parsed_tool.
2. Verify that the tool exists and is callable.
3. Execute the tool with the extracted arguments.
4. Capture the tool's output and return it.

In [26]:
# Extract tool name and arguments
tool_name = parsed_tool["name"]
arguments = parsed_tool["arguments"]

# Available tools (mapping tool names to functions)
available_tools = {
    "calculator": calculator
}

# Verify tool exists
if tool_name not in available_tools:
    raise ValueError(f"Unknown tool: {tool_name}. Available tools: {list(available_tools.keys())}")

# Execute tool
tool_function = available_tools[tool_name]
result = tool_function(**arguments)

print(f"Tool Execution Result: {result}")

Tool Execution Result: 30
